# Libraries

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
!pip install contractions 

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import math
import contractions
from collections import Counter

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download('omw-1.4')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [ ]:
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn import metrics

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

# Reading data

In [ ]:
path = "/content/gdrive/MyDrive/dep-nlp/"

In [ ]:
train_data = pd.read_csv(path+'data/train.csv')
train_data.head()

In [ ]:
# train_data = train_data.rename(columns={'PID': 'id', 'Text_data': 'text', 'Label': 'label'})

In [ ]:
test_data = pd.read_csv(path+'data/test.csv')
test_data.head()

In [ ]:
# test_data = test_data.rename(columns={'Pid': 'id', 'text data': 'text'})

In [ ]:
dev_data = pd.read_csv(path+'data/dev.csv')
dev_data.head()

In [ ]:
# dev_data = dev_data.rename(columns={'PID': 'id', 'Text data': 'text', 'Label': 'label'})

In [ ]:
train_data.to_csv('train.csv',index=False)
dev_data.to_csv('dev.csv', index=False)
test_data.to_csv('test.csv', index=False)

In [ ]:
possible_labels = train_data.label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

In [ ]:
train_data['label'] = train_data.label.replace(label_dict)
dev_data['label'] = dev_data.label.replace(label_dict)

In [ ]:
from collections import Counter

print('train')
print(Counter(train_data['label']).keys()) # unique elements
print(Counter(train_data['label']).values()) # counts the elements' frequency

print('dev')
print(Counter(dev_data['label']).keys()) # unique elements
print(Counter(dev_data['label']).values()) # counts the elements' frequency

In [ ]:
dev_data.head()

# Preprocessing

In [ ]:
def preprocess(text):

  text=contractions.fix(text) # expanding the contractions
  text=text.lower() # lowering the text
  text=re.sub(r'[^\w\s]', '', text) # removing the punctuation
  text=''.join([i for i in text if not i.isdigit()]) # removing the digits and words containing the digits
  text=" ".join([word for word in str(text).split() if word not in stop_words]) # removing the stopwords
  text=" ".join([lemmatizer.lemmatize(word) for word in text.split()]) # lemmatizing the words

  return text

In [ ]:
train_data['text'] = train_data['text'].apply(lambda x: preprocess(x))

In [ ]:
test_data['text'] = test_data['text'].apply(lambda x: preprocess(x))

In [ ]:
dev_data['text'] = dev_data['text'].apply(lambda x: preprocess(x))

In [ ]:
# train_data.iloc[0]['text']

In [ ]:
train_data.to_csv('train-preprocess.csv')
test_data.to_csv('test-preprocess.csv')
dev_data.to_csv('dev-preprocess.csv')

In [ ]:
train_data.head()

# Generate embeddings

In [ ]:
from transformers import AutoTokenizer, AutoModel

### Mental-bert
https://huggingface.co/mental/mental-bert-base-uncased?text=The+goal+of+life+is+%5BMASK%5D.

In [ ]:
tokenizer1 = AutoTokenizer.from_pretrained("mental/mental-bert-base-uncased")
model1 = AutoModel.from_pretrained("mental/mental-bert-base-uncased")

### MentalRoBERTa
https://huggingface.co/mental/mental-roberta-base?text=The+goal+of+life+is+%3Cmask%3E.

In [ ]:
tokenizer2 = AutoTokenizer.from_pretrained("mental/mental-roberta-base")
model2 = AutoModel.from_pretrained("mental/mental-roberta-base")

### Bio+Clinical BERT
https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT?text=Paris+is+the+%5BMASK%5D+of+France.

In [ ]:
tokenizer3 = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model3 = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

Looping for embeddings:


* embedding1-label1: mental-bert
* embedding2-label2: mental-roberta
* embedding3-label3: bioclinical-bert



In [ ]:
embedding1 = []
label1=[]
embedding2 = []
label2=[]
embedding3 = []
label3 = []

In [ ]:
counter = 0

for i, j in zip(dev_data['text'], dev_data['label']):
  if(counter%500==0):print(counter)
  counter = counter+1
  try:
    input1 = tokenizer1(i, return_tensors="pt", padding=True, truncation=True)
    output1 = model1(**input1)

    input2 = tokenizer2(i, return_tensors='pt', padding=True, truncation=True)
    output2 = model2(**input2)

    input3 = tokenizer3(i, return_tensors="pt", padding=True, truncation=True)
    output3 = model3(**input3)

    embedding1.append(output1.last_hidden_state[0][0].detach().numpy())
    embedding2.append(output2.last_hidden_state[0][0].detach().numpy())
    embedding3.append(output3.last_hidden_state[0][0].detach().numpy())
    label1.append(j)
    label2.append(j)
    label3.append(j)
    # print(outputs.last_hidden_state.shape)
  except:
    continue

In [ ]:
print(len(label1), len(label2), len(label3))

In [ ]:
print(len(embedding1))

In [ ]:
print(len(embedding1[0]))

In [ ]:
np.save(open('dev-mental-bert-emb.npy','wb'),np.array(embedding1))
np.save(open('dev-mental-roberta-emb.npy','wb'),np.array(embedding2))
np.save(open('dev-clinical-bert-emb.npy','wb'),np.array(embedding3))

In [ ]:
np.save(open('dev-mental-bert-label.npy','wb'),np.array(label1))
np.save(open('dev-mental-roberta-label.npy','wb'),np.array(label2))
np.save(open('dev-clinical-bert-label.npy','wb'),np.array(label3))

In [ ]:
print(label1)

In [ ]:
np.save(open('dev-stack-label.npy','wb'),np.array(label1))

In [ ]:
# stacking the embeddings
stack = [sum(x)/ 3 for x in zip(embedding1, embedding2, embedding3)]

In [ ]:
print(type(stack))
print(len(stack))

In [ ]:
np.save(open('dev-stack-emb.npy','wb'),np.array(stack))

### Checking occurences

In [ ]:
print(Counter(label1).keys()) # unique elements
print(Counter(label1).values()) # counts the elements' frequency

### Some data modifications

In [ ]:
# train_emb = np.load(path+'emb/train/train-stack-emb.npy')
# train_label = np.load(path+'emb/train/train-clinical-bert-label.npy')

In [ ]:
# dev_emb = np.load(path+'emb/dev/dev-stack-emb.npy')
# dev_label = np.load(path+'emb/dev/dev-stack-label.npy')

In [ ]:
# np.unique(train_label)

In [ ]:
# type(train_label)

In [ ]:
# train_label = np.load(path+'emb/train/train-mental-roberta-label.npy')

In [ ]:
'''
def convert_labels(arr):
    for i in range(len(arr)):
        if arr[i]=='moderate':
            arr[i]=0
        elif arr[i]=='not depression':
            arr[i]=1
        elif arr[i]=='severe':
            arr[i]=2

convert_labels(train_label)
'''


In [ ]:
# np.unique(train_label)

In [ ]:
# train_label = train_label.astype(np.int64)

In [ ]:
# np.unique(train_label)

In [ ]:
# np.save('train-mental-roberta-label.npy', np.array(train_label))

In [ ]:
# np.unique(dev_label)

# SVM

### For stacked embeddings

In [ ]:
train_emb = np.load(path+'emb/train/train-stack-emb.npy')
train_label = np.load(path+'emb/train/train-stack-label.npy')

In [ ]:
dev_emb = np.load(path+'emb/dev/dev-stack-emb.npy')
dev_label = np.load(path+'emb/dev/dev-stack-label.npy')

In [ ]:
svm_classifier = OneVsRestClassifier(SVC())
svm_classifier.fit(train_emb, train_label)

In [ ]:
pred = svm_classifier.predict(dev_emb)

In [ ]:
print("Accuracy for stacked embeddings (without balancing)")
# Model Accuracy: how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(pred, dev_label))
print(classification_report(pred, dev_label))

# Balancing using SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE
smt = SMOTE()

In [ ]:
train_emb_smt, train_label_smt = smt.fit_resample(train_emb, train_label)

In [ ]:
print('Before:', Counter(train_label))
print('After:', Counter(train_label_smt))

In [ ]:
svm_classifier.fit(train_emb_smt, train_label_smt)

In [ ]:
pred_bal = svm_classifier.predict(dev_emb)

In [ ]:
print("Accuracy for stacked embeddings (after balancing)")
# Model Accuracy: how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(pred_bal, dev_label))
print(classification_report(pred_bal, dev_label))

### For mental-bert embeddings

In [ ]:
train_emb1 = np.load(path+'emb/train/train-mental-bert-emb.npy')
train_label1 = np.load(path+'emb/train/train-mental-bert-label.npy')

In [ ]:
dev_emb1 = np.load(path+'emb/dev/dev-mental-bert-emb.npy')
dev_label1 = np.load(path+'emb/dev/dev-mental-bert-label.npy')

In [ ]:
svm_classifier1 = OneVsRestClassifier(SVC())
svm_classifier1.fit(train_emb1, train_label1)

In [ ]:
pred1 = svm_classifier1.predict(dev_emb1)

In [ ]:
print("Accuracy for mental-bert embeddings (without balancing)")
# Model Accuracy: how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(pred1, dev_label1))
print(classification_report(pred1, dev_label1))

In [ ]:
train_emb_smt1, train_label_smt1 = smt.fit_resample(train_emb1, train_label1)

In [ ]:
svm_classifier.fit(train_emb_smt1, train_label_smt1)

In [ ]:
pred_bal1 = svm_classifier.predict(dev_emb1)

In [ ]:
print("Accuracy for stacked embeddings (after balancing)")
# Model Accuracy: how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(pred_bal1, dev_label1))
print(classification_report(pred_bal1, dev_label1))